In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing packages

In [2]:
import pandas as pd
import networkx as nx

In [3]:
# !ls "/content/drive/MyDrive/Network Science Project Files"


In [4]:
file_path = "/content/drive/MyDrive/Network Science Project Files"

In [5]:
df = pd.read_csv(file_path+'/all_records.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (20,21,22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# df.head()

In [7]:
df.columns = list(map(str.strip, df.columns))

In [8]:
df['NORMED?'] = df['NORMED?'].str.strip()
df['CUE'] = df['CUE'].str.strip()
df['TARGET'] = df['TARGET'].str.strip()

In [9]:
df = df[df['NORMED?']=='YES']

In [10]:
df = df[['CUE','TARGET','NORMED?','#G','#P','FSG','BSG','QPS','TPS']]

* CUE	Normed Word
* TARGET	Response to Normed Word
* NORMED?	Is Response Normed?
* \#G	Group size
* \#P	Number of Participants Producing Response
* FSG	Forward Cue-to-Target Strength
* BSG	Backward Target-to-Cue Strength
* QPS	Cue: Part of Speech
* TPS	Target: Part of Speech





In [11]:
## filtering POS for CUES and TARGETS to be, nouns verbs and adjectives
exclude_list = ['PP','AV','ADV','P','I','C']
df1 = df[~df['QPS'].str.contains('|'.join(exclude_list))]
df1['TPS'] = df1['TPS'].dropna()
df1 = df1[df1['TPS']!=' ']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


* N	Noun
* V	Verb
* AJ	Adjective
* AD	Adverb
* P	Pronoun
* PP	Preposition
* I	Interjection
* C	Conjunction

In [12]:
df1['BSG'] = df1['BSG'].astype(float)

Filtering BSG to be 0 and the number of participants to be more than 2

In [13]:
df1_n_bsg = df1[df1['BSG']==0]
df1_n_bsg = df1_n_bsg[df1_n_bsg['#P']>2]

## Graph for the Cues starting with letter A

In [14]:
# df1_n_bsg_a_b = df1_n_bsg[(df1_n_bsg['CUE'].str.startswith('A')) | (df1_n_bsg['CUE'].str.startswith('B'))]
df1_n_bsg_a = df1_n_bsg[(df1_n_bsg['CUE'].str.startswith('A'))]

In [15]:
lst = list(set(list(df1_n_bsg_a['CUE'])+list(df1_n_bsg_a['TARGET'])))

In [17]:
G = nx.Graph()
for n in lst:
  G.add_node(n)
for row in df1_n_bsg_a.itertuples():
  # print(row.CUE)
  G.add_edge(row.CUE,row.TARGET)

In [ ]:
nx.write_gml(G,'cue_target.gml')

## Generating Graph for the the Parts of Speech of the Cues and the Target - Cues are already filtered for Nouns, Verbs and Adjectives

In [22]:
df1

,CUE,TARGET,NORMED?,#G,#P,FSG,BSG,QPS,TPS
1,A,ALPHABET,YES,152,10,0.066,0.046,N,N
3,A,GRADE,YES,152,9,0.059,0.277,N,N
4,A,LETTER,YES,152,6,0.039,0.000,N,N
5,A,PLUS,YES,152,5,0.033,0.000,N,PP
9,A,APPLE,YES,152,2,0.013,0.000,N,V
...,...,...,...,...,...,...,...,...,...
63819,ZUCCHINI,BREAD,YES,148,2,0.014,0.000,N,N
63821,ZUCCHINI,FRUIT,YES,148,2,0.014,0.000,N,N
63822,ZUCCHINI,GROSS,YES,148,2,0.014,0.000,N,AJ
63823,ZUCCHINI,ITALIAN,YES,148,2,0.014,0.000,N,AJ


In [84]:
df1['TPS'] = df1['TPS'].astype('object')
df1['TPS'] = df1['TPS'].str.strip()


In [85]:
df1.dtypes

CUE         object
TARGET      object
NORMED?     object
#G           int64
#P           int64
FSG        float64
BSG        float64
QPS         object
TPS         object
dtype: object

In [93]:
df1['TPS'].value_counts()

N      36211
AJ      9153
V       8980
AD       860
ADJ      240
P        184
PP       128
I         78
AV        11
INT        9
C          8
ADV        6
PRP        4
Name: TPS, dtype: int64

In [87]:
df1 = df1.dropna()

In [88]:
df1['TPS'] = df1['TPS'].astype(str)

In [89]:
include_list = ['N','V','AJ','AD','P','PP','I','C']
df1 = df1[df1['TPS'].str.contains('|'.join(include_list))]

In [101]:
d = {}
for row in df1.itertuples():
  # print(row.TPS)
  if row.QPS not in d:
    if row.TPS in ['N','AJ','V','AD','P','PP','I','C']: 
      d[row.QPS] = [row.TPS]
    else:
      continue
  elif row.TPS in ['N','AJ','V','AD','P','PP','I','C']: d[row.QPS].append(row.TPS)


In [104]:
lst = ['N','AJ','V','AD','P','PP','I','C']

In [107]:
G = nx.Graph(d)

In [108]:
nx.write_gml(G,'POS.gml')